In [1]:
import os
import sys

In [2]:
# Update env and pkg paths.

sys.path.extend(['/data/xzuo/.conda/envs/dissertation/lib/python38.zip', '/data/xzuo/.conda/envs/dissertation/lib/python3.8', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/lib-dynload', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/site-packages'])
print(sys.path)

['/data/xzuo/Projects/dissertation/dataset_URL_parser/paper_oriented', '/usr/local/anaconda3/lib/python39.zip', '/usr/local/anaconda3/lib/python3.9', '/usr/local/anaconda3/lib/python3.9/lib-dynload', '', '/home/xzuo/.local/lib/python3.9/site-packages', '/usr/local/anaconda3/lib/python3.9/site-packages', '/usr/local/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/usr/local/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/xzuo/.ipython', '/data/xzuo/.conda/envs/dissertation/lib/python38.zip', '/data/xzuo/.conda/envs/dissertation/lib/python3.8', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/lib-dynload', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/site-packages']


## Load citation links

In [3]:
import pubmed_parser as pp
import pandas as pd
import re
from lxml import etree

In [4]:
# Get pmid from LitCovid.

id_list_path = "./litcovid_pmid/litcovid_16052022.tsv"
csv_directory_path = "./litcovid_pmcid_lists"
download_directory_path ="./litcovid_pmc_fulltext/xml"

current_filelist = os.listdir(download_directory_path)

In [5]:
# Remove inaccessible files.

inaccessible_articles = []

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    try:
        pmc_tree = etree.parse(pmc_xml_path)
    except:
        # print("Remove file %s." % xml_name)
        inaccessible_articles.append(xml_name)
        current_filelist.remove(xml_name)

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    pmc_tree = etree.parse(pmc_xml_path)
    meta = pmc_tree.find(".//article-meta")
    if meta is None:
        inaccessible_articles.append(xml_name)
        current_filelist.remove(xml_name)
#         print("File %s is not accessible." % xml_name)
    else:
        continue

print("There are %d articles not accessible." % len(inaccessible_articles))

for i, xml_name in enumerate(inaccessible_articles):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    if os.path.exists(pmc_xml_path):
        os.remove(pmc_xml_path)
    else:
        print("Error: File %s does not exist." % xml_name)

There are 0 articles not accessible.


In [6]:
from utils import read_xml, stringify_children
from pubmed_oa_parser import parse_article_meta

In [7]:
def parse_pubmed_references(path):
    """
    Given path to xml file, parse references articles
    to list of dictionary
    Parameters
    ----------
    path: str
        A string to an XML path.
    Return
    ------
    dict_refs: list
        A list contains dictionary for references made in a given file.
    """
    tree = read_xml(path)
    dict_article_meta = parse_article_meta(tree)
    pmid = dict_article_meta["pmid"]
    pmc = dict_article_meta["pmc"]

    references = tree.xpath(".//ref-list/ref[@id]")
    dict_refs = list()
    for reference in references:
        ref_id = reference.attrib["id"]

        if reference.find("mixed-citation") is not None:
            ref = reference.find("mixed-citation")
        elif reference.find("element-citation") is not None:
            ref = reference.find("element-citation")
        else:
            ref = None

        if ref is not None:
            if "publication-type" in ref.attrib.keys() and ref is not None:
                if ref.attrib.values() is not None:
                    journal_type = ref.attrib.values()[0]
                else:
                    journal_type = ""
                names = list()
                if ref.find("name") is not None:
                    for n in ref.findall("name"):
                        name = " ".join([t.text or "" for t in n.getchildren()][::-1])
                        names.append(name)
                elif ref.find("person-group") is not None:
                    for n in ref.find("person-group"):
                        name = " ".join(
                            n.xpath("given-names/text()") + n.xpath("surname/text()")
                        )
                        names.append(name)
                if ref.find("article-title") is not None:
                    article_title = stringify_children(ref.find("article-title")) or ""
                    article_title = article_title.replace("\n", " ").strip()
                else:

                    article_title = ""
                if ref.find("source") is not None:
                    journal = ref.find("source").text or ""
                else:
                    journal = ""
                if ref.find("year") is not None:
                    year = ref.find("year").text or ""
                else:
                    year = ""
                if len(ref.findall("pub-id")) >= 1:
                    for pubid in ref.findall("pub-id"):
                        if "doi" in pubid.attrib.values():
                            doi_cited = pubid.text
                        else:
                            doi_cited = ""
                        if "pmid" in pubid.attrib.values():
                            pmid_cited = pubid.text
                        else:
                            pmid_cited = ""
                else:
                    doi_cited = ""
                    pmid_cited = ""
                if ref.find("uri") is not None:
                    url = ref.find("uri").text
                else:
                    url = ""
                dict_ref = {
                    "pmid": pmid,
                    "pmc": pmc,
                    "ref_id": ref_id,
                    "pmid_cited": pmid_cited,
                    "doi_cited": doi_cited,
                    "article_title": article_title,
                    "name": "; ".join(names),
                    "year": year,
                    "journal": journal,
                    "journal_type": journal_type,
                    "url": url,
                }
                dict_refs.append(dict_ref)
    if len(dict_refs) == 0:
        dict_refs = None
    return dict_refs

In [8]:
# Parse out abstract and citation links.
articles = []

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    pmc_tree = etree.parse(pmc_xml_path)
    meta = pmc_tree.find(".//article-meta")
    #if xml_name == "PMC8675556.xml" :
    try:
        metadata = pp.parse_pubmed_xml(pmc_xml_path)
        ref_dict = parse_pubmed_references(pmc_xml_path)
        pmc_article = {"pmid": metadata['pmid'], "pmcid": metadata['pmc'], 
                       "title": metadata['full_title'], "abstract": metadata['abstract'], 
                       "references": ref_dict}
        articles.append(pmc_article)
    except:
        os.remove(pmc_xml_path)
        print("File %s is not accessible. Removed." % xml_name)

df = pd.DataFrame(articles)
display(df)

,pmid,pmcid,title,abstract,references
0,34776649,8572530,Subjektives Befinden und Arbeitsfähigkeit nach...,Hintergrund Seit dem 29.01.2021 wurde der C...,"[{'pmid': '34776649', 'pmc': '8572530', 'ref_i..."
1,34098882,8182346,"Web-based, rapid and contactless management of...",Background During the SARS-CoV-2 pandemic a...,"[{'pmid': '34098882', 'pmc': '8182346', 'ref_i..."
2,34541298,8441688,Molecular Level Dissection of Critical Spike M...,Abstract SARS‐CoV‐2 virus during its spread ...,"[{'pmid': '34541298', 'pmc': '8441688', 'ref_i..."
3,32302815,7151272,Psychological status of surgical staff during ...,Highlight • The mental health problems o...,"[{'pmid': '32302815', 'pmc': '7151272', 'ref_i..."
4,34295171,8291382,The Immunogenicity of Hyaluronic Fillers and I...,Abstract Hyaluronic acid (HA) is a glycosami...,"[{'pmid': '34295171', 'pmc': '8291382', 'ref_i..."
...,...,...,...,...,...
74580,33564622,7862904,WITHDRAWN: Automatic optimized CNN based COVID...,This article has been withdrawn: please see E...,None
74581,34736267,8678245,Psychosocial Factors Predict COVID-19 Vaccine ...,,"[{'pmid': '34736267', 'pmc': '8678245', 'ref_i..."
74582,33100647,7573996,Anesthesiologist and COVID-19—current perspective,In the current situation when the world is gr...,"[{'pmid': '33100647', 'pmc': '7573996', 'ref_i..."
74583,34343282,8385824,Improving Pandemic Response: Employing Mathema...,Abstract Modeling complements surveillance d...,None


## Search datasets

In [ ]:
name_pattern = re.compile(r'(N3C)|(National COVID Cohort Collaborative)|(GenBank)|\
                     (GISAID)|(Nextstrain)|(OpenICPSR)|(CORD-19)|(LitCovid)|(NCBI Virus)')
url_pattern = re.compile(r'')

In [ ]:
for i, article in enumerate(articles):
    title_abstract_text = article['title'] + ".\n" + article['abstract']
    article['sentences'] = seg.segment(title_abstract_text)
    article['name_match'] = []
    for j, sent in enumerate(article['sentences']):
        if pattern.search(sent):
            article['has_pattern'].append(sent)
    if i != 0 and i % 1000 == 0:
        print("Searched %d full-text articles." % i)
        # break

print("Search Completed.")